<a href="https://colab.research.google.com/github/Parthi1212-dotcom/Investment-Portfolio-through-Evolutionary-algorithms/blob/main/Binomial_Model_with_GA_(Yahoo_Finance).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import numpy as np
import datetime
import random
import time

# --- 1. Binomial Options Pricing Model (Cox-Ross-Rubinstein) ---
def binomial_option_pricing(S, K, T, r, sigma, n=100, option_type='call'):
    """
    Calculates the European option price using the Cox-Ross-Rubinstein binomial model.

    Args:
        S (float): Current stock price.
        K (float): Strike price of the option.
        T (float): Time to expiration in years.
        r (float): Risk-free interest rate (annual).
        sigma (float): Volatility of the underlying stock (annual).
        n (int): Number of steps in the binomial tree.
        option_type (str): Type of the option, 'call' or 'put'.

    Returns:
        float: The calculated price of the option.
    """
    # Calculate binomial tree parameters
    dt = T / n  # Time step
    u = np.exp(sigma * np.sqrt(dt))  # Up-factor
    d = 1 / u  # Down-factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    # Initialize asset prices at maturity (at the end of the tree)
    asset_prices = np.zeros(n + 1)
    for i in range(n + 1):
        asset_prices[i] = S * (u**(n - i)) * (d**i)

    # Initialize option values at maturity based on the payoff
    option_values = np.zeros(n + 1)
    if option_type == 'call':
        option_values = np.maximum(0, asset_prices - K)
    else:  # 'put'
        option_values = np.maximum(0, K - asset_prices)

    # Work backwards through the tree to find the option price today
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            # Discounted expected value of the option in the next step
            option_values[j] = np.exp(-r * dt) * (p * option_values[j] + (1 - p) * option_values[j + 1])

    return option_values[0]

# --- 2. Genetic Algorithm for Finding Implied Volatility ---
def fitness_function(volatility, target_price, S, K, T, r, n, option_type):
    """
    Calculates the fitness of a given volatility value.
    Fitness is inversely proportional to the pricing error.
    A higher fitness score is better.
    """
    # Ensure volatility is positive, otherwise, it's a useless guess
    if volatility <= 0.001:
        return 0

    # Calculate the option price using the binomial model with the guessed volatility
    model_price = binomial_option_pricing(S, K, T, r, volatility, n, option_type)

    # Calculate the error (difference between model and market price)
    error = abs(model_price - target_price)

    # The fitness is the inverse of the error. We add 1 to avoid division by zero.
    return 1 / (1.0 + error)

def genetic_algorithm(target_price, S, K, T, r, n, option_type, population_size=100, generations=50, crossover_rate=0.8, mutation_rate=0.2):
    """
    Finds the implied volatility for an option using a Genetic Algorithm.

    Args:
        target_price (float): The market price of the option we are targeting.
        All other args are passed to the binomial model.
        population_size (int): Number of individuals (volatility guesses) in each generation.
        generations (int): Number of generations to evolve.
        crossover_rate (float): Probability of two parents creating offspring.
        mutation_rate (float): Probability of a random mutation in an individual.

    Returns:
        float: The best volatility found by the GA.
    """
    # 1. Initialization: Create an initial population of random volatility guesses.
    # Volatility is typically between 5% and 200%.
    population = [random.uniform(0.05, 2.0) for _ in range(population_size)]

    for gen in range(generations):
        # 2. Fitness Evaluation: Calculate how "good" each volatility guess is.
        fitness_scores = [fitness_function(vol, target_price, S, K, T, r, n, option_type) for vol in population]

        # 3. Selection: Choose parents for the next generation.
        # Individuals with higher fitness are more likely to be chosen (Roulette Wheel Selection).
        total_fitness = sum(fitness_scores)
        if total_fitness == 0: # Avoid division by zero if all fitness is 0
            population = [random.uniform(0.05, 2.0) for _ in range(population_size)]
            continue

        selection_probs = [score / total_fitness for score in fitness_scores]

        new_population = []
        for _ in range(population_size // 2):
            # Select two parents based on their fitness probability
            parent1 = random.choices(population, weights=selection_probs, k=1)[0]
            parent2 = random.choices(population, weights=selection_probs, k=1)[0]

            # 4. Crossover: Create two children from the parents.
            if random.random() < crossover_rate:
                # Simple arithmetic mean for crossover
                child1 = (parent1 + parent2) / 2
                child2 = (parent1 + parent2) / 2
            else:
                # If no crossover, children are clones of parents
                child1, child2 = parent1, parent2

            # 5. Mutation: Introduce small random changes to the children.
            if random.random() < mutation_rate:
                child1 += random.uniform(-0.05, 0.05)
            if random.random() < mutation_rate:
                child2 += random.uniform(-0.05, 0.05)

            new_population.extend([child1, child2])

        population = new_population

    # After all generations, find the best individual in the final population
    final_fitness_scores = [fitness_function(vol, target_price, S, K, T, r, n, option_type) for vol in population]
    best_individual_index = np.argmax(final_fitness_scores)

    return population[best_individual_index]


# --- 3. Main Execution Block ---
if __name__ == '__main__':
    # --- User-Defined Parameters ---
    ticker_symbol = 'NVDA'
    expiry_date = '2026-01-16' # Format: YYYY-MM-DD
    option_type = 'call'       # 'call' or 'put'

    print(f"--- Starting Analysis for {ticker_symbol} ---")

    try:
        # --- Data Fetching from Yahoo Finance ---
        stock = yf.Ticker(ticker_symbol)

        # Get current stock price (most recent closing price)
        current_stock_price = stock.history(period='1d')['Close'].iloc[0]

        # Get options chain for the specified expiry date
        opt = stock.option_chain(expiry_date)
        options_df = opt.calls if option_type == 'call' else opt.puts

        if options_df.empty:
            raise ValueError(f"No {option_type} options found for {ticker_symbol} on {expiry_date}.")

        # Select an option to analyze: the one closest to the current stock price (at-the-money)
        at_the_money_option = options_df.iloc[(options_df['strike'] - current_stock_price).abs().argsort()[:1]]

        if at_the_money_option.empty:
            raise ValueError("Could not find an at-the-money option.")

        market_price = at_the_money_option['lastPrice'].iloc[0]
        strike_price = at_the_money_option['strike'].iloc[0]

        # --- Set Up Model Parameters ---
        # Time to expiration (in years)
        days_to_expiry = (datetime.datetime.strptime(expiry_date, "%Y-%m-%d") - datetime.datetime.now()).days
        T = days_to_expiry / 365.25

        # Risk-free rate (using 10-year Treasury yield as a proxy, fetched from Yahoo Finance)
        # ^TNX is the ticker for the 10-Year Treasury Note Yield
        treasury_yield_ticker = yf.Ticker('^TNX')
        r = treasury_yield_ticker.history(period='1d')['Close'].iloc[0] / 100

        # Binomial tree steps (higher n = more accuracy, but slower computation)
        n_steps = 100

        # --- Print Analysis Setup ---
        print(f"\n--- Analyzing {ticker_symbol} {option_type.capitalize()} Option ---")
        print(f"Current Stock Price (S): {current_stock_price:.2f}")
        print(f"Strike Price (K): {strike_price:.2f}")
        print(f"Market Option Price: {market_price:.2f}")
        print(f"Expiration Date: {expiry_date} ({days_to_expiry} days from now)")
        print(f"Time to Expiration (T): {T:.3f} years")
        print(f"Risk-Free Rate (r): {r:.4f} ({r*100:.2f}%)")
        print("---------------------------------------------")
        print("Running Genetic Algorithm to find Implied Volatility...")
        start_time = time.time()

        # --- Run the GA to find implied volatility ---
        implied_volatility = genetic_algorithm(
            target_price=market_price,
            S=current_stock_price,
            K=strike_price,
            T=T,
            r=r,
            n=n_steps,
            option_type=option_type,
            population_size=150, # Fine-tune these GA params if needed
            generations=60
        )

        end_time = time.time()
        print(f"GA search completed in {end_time - start_time:.2f} seconds.")

        # --- Verification and Results ---
        print(f"\n--- Results ---")
        print(f"Implied Volatility (σ) found by GA: {implied_volatility:.4f} ({implied_volatility*100:.2f}%)")

        # Price the option with the GA's discovered volatility to verify
        model_price = binomial_option_pricing(current_stock_price, strike_price, T, r, implied_volatility, n_steps, option_type)
        print(f"Binomial Model Price with this volatility: {model_price:.2f}")
        print(f"Original Market Price: {market_price:.2f}")
        print(f"Pricing Error: {abs(model_price - market_price):.4f}")

    except Exception as e:
        print(f"\nAn error occurred: {e}")
        print("Please check the ticker symbol and expiration date. Ensure you have an internet connection.")

--- Starting Analysis for NVDA ---

--- Analyzing NVDA Call Option ---
Current Stock Price (S): 171.93
Strike Price (K): 172.00
Market Option Price: 20.80
Expiration Date: 2026-01-16 (184 days from now)
Time to Expiration (T): 0.504 years
Risk-Free Rate (r): 0.0447 (4.47%)
---------------------------------------------
Running Genetic Algorithm to find Implied Volatility...
GA search completed in 68.93 seconds.

--- Results ---
Implied Volatility (σ) found by GA: 0.3938 (39.38%)
Binomial Model Price with this volatility: 20.80
Original Market Price: 20.80
Pricing Error: 0.0009
